In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [3]:
X,Y = load_coffee_data();
print(X.shape, Y.shape)

(200, 2) (200, 1)


In [5]:
norm_layer = tf.keras.layers.Normalization(axis=-1)
norm_layer.adapt(X)
x_norm = norm_layer(X)

In [6]:
print(f"Temperature Max, Min pre normalization: {np.max(X[:,0]):0.2f}, {np.min(X[:,0]):0.2f}")
print(f"Duration    Max, Min pre normalization: {np.max(X[:,1]):0.2f}, {np.min(X[:,1]):0.2f}")

Temperature Max, Min pre normalization: 284.99, 151.32
Duration    Max, Min pre normalization: 15.45, 11.51


In [7]:
print(f"Temperature Max, Min post normalization: {np.max(x_norm[:,0]):0.2f}, {np.min(x_norm[:,0]):0.2f}")
print(f"Duration    Max, Min post normalization: {np.max(x_norm[:,1]):0.2f}, {np.min(x_norm[:,1]):0.2f}")

Temperature Max, Min post normalization: 1.66, -1.69
Duration    Max, Min post normalization: 1.79, -1.70


In [13]:
Xt = np.tile((x_norm),(1000,1))
Yt= np.tile(Y,(1000,1))   

### Creating a 2 layer NN 
#### input layer with 2 values, layer1 (hidden) with 3 units and layer2 (output) with 1 unit

In [11]:
model = Sequential(
    [
        tf.keras.Input(shape=(2,)),
        Dense(units=3, activation = 'sigmoid', name = 'layer1'),
        Dense(units=1, activation = 'sigmoid', name = 'layer2')
    ]
)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 9         
                                                                 
 dense_1 (Dense)             (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


In [12]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1)
print(f"W2{W2.shape}:\n", W2, f"\nb2{b2.shape}:", b2)

W1(2, 3):
 [[ 0.77160895  0.07172191  0.7373843 ]
 [ 0.9463682  -0.8324934  -0.48892725]] 
b1(3,): [0. 0. 0.]
W2(3, 1):
 [[1.1459578]
 [0.26616  ]
 [1.1822165]] 
b2(1,): [0.]


In [14]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 18s 2ms/step - loss: 0.2258
Epoch 2/10
6250/6250 [==============================] - 16s 2ms/step - loss: 0.1456
Epoch 3/10
6250/6250 [==============================] - 17s 3ms/step - loss: 0.1384
Epoch 4/10
6250/6250 [==============================] - 15s 2ms/step - loss: 0.1356
Epoch 5/10
6250/6250 [==============================] - 16s 3ms/step - loss: 0.1338
Epoch 6/10
6250/6250 [==============================] - 17s 3ms/step - loss: 0.1328
Epoch 7/10
6250/6250 [==============================] - 17s 3ms/step - loss: 0.1321
Epoch 8/10
6250/6250 [==============================] - 18s 3ms/step - loss: 0.1315
Epoch 9/10
6250/6250 [==============================] - 18s 3ms/step - loss: 0.1311
Epoch 10/10
6250/6250 [==============================] - 19s 3ms/step - loss: 0.1307


In [15]:
W1, b1 = model.get_layer("layer1").get_weights()
W2, b2 = model.get_layer("layer2").get_weights()
print("W1:\n", W1, "\nb1:", b1)
print("W2:\n", W2, "\nb2:", b2)

W1:
 [[  8.7769375  10.9003      6.5485044]
 [  2.715849    7.975337  -16.246786 ]] 
b1: [12.088926  -1.4403552 19.64502  ]
W2:
 [[  9.809029]
 [-22.780975]
 [  7.118369]] 
b2: [-14.61966]


In [16]:
W1 = np.array([
    [-8.94,  0.29, 12.89],
    [-0.17, -7.34, 10.79]] )
b1 = np.array([-9.87, -9.28,  1.01])
W2 = np.array([
    [-31.38],
    [-27.86],
    [-32.79]])
b2 = np.array([15.54])
model.get_layer("layer1").set_weights([W1,b1])
model.get_layer("layer2").set_weights([W2,b2])

In [18]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_layer(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 316ms/step
predictions = 
 [[9.625139e-01]
 [3.031606e-08]]


In [19]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [20]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[1]
 [0]]
